# The model:

In [4]:
!pip install -q pandas scikit-learn joblib tldextract

from google.colab import files
uploaded = files.upload()

import pandas as pd
import re
import tldextract

# Replace this filename with your uploaded CSV file if different
csv_file = list(uploaded.keys())[0]  # Automatically pick uploaded file
df = pd.read_csv(csv_file)

# ✅ Step 4: Preprocess data
# Keep only relevant columns (assumes 'URL' and 'Label' columns exist)
df = df[['url', 'type']].rename(columns={'url': 'url', 'type': 'label'})
df = df.dropna()
df['label'] = df['label'].apply(lambda x: 1 if str(x).lower() == 'malicious' else 0)

# ✅ Step 5: Feature extraction
def extract_features(url):
    ext = tldextract.extract(url)
    domain = ext.domain
    suffix = ext.suffix
    return {
        "url_length": len(url),
        "hostname_length": len(domain),
        "has_https": int("https" in url),
        "num_dots": url.count('.'),
        "num_hyphens": url.count('-'),
        "num_digits": sum(c.isdigit() for c in url),
        "has_ip": int(bool(re.search(r'\d+\.\d+\.\d+\.\d+', url))),
        "has_at_symbol": int("@" in url),
        "has_exe": int(".exe" in url),
        "suffix_length": len(suffix)
    }

# Apply feature extraction
df['features'] = df['url'].apply(extract_features)
features_df = pd.DataFrame(df['features'].tolist())
X = features_df
y = df['label']

# ✅ Step 6: Train the Random Forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# ✅ Step 7: Evaluate the model
print("✅ Model Accuracy:", clf.score(X_test, y_test))

# ✅ Step 8: Export the model
import joblib
joblib.dump(clf, "rf_url_model.pkl")

# ✅ Step 9: Download the model to your local system
files.download("rf_url_model.pkl")


Saving malicious_phish.csv to malicious_phish (1).csv
✅ Model Accuracy: 1.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# The app:

In [7]:
import streamlit as st
import pandas as pd
import re
import tldextract
import joblib
import sqlite3
from io import BytesIO

DB_PATH = "urls.db"

# ================================
# Feature Extraction
# ================================
def extract_features(url):
    ext = tldextract.extract(url)
    domain = ext.domain
    suffix = ext.suffix
    return pd.DataFrame([{
        "url_length": len(url),
        "hostname_length": len(domain),
        "has_https": int("https" in url),
        "num_dots": url.count('.'),
        "num_hyphens": url.count('-'),
        "num_digits": sum(c.isdigit() for c in url),
        "has_ip": int(bool(re.search(r'\d+\.\d+\.\d+\.\d+', url))),
        "has_at_symbol": int("@" in url),
        "has_exe": int(".exe" in url),
        "suffix_length": len(suffix)
    }])

# ================================
# SQLite Functions
# ================================
def init_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS malicious_urls (id INTEGER PRIMARY KEY, url TEXT UNIQUE)")
    conn.commit()
    conn.close()

def insert_url(url):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("INSERT OR IGNORE INTO malicious_urls (url) VALUES (?)", (url,))
    conn.commit()
    conn.close()

def get_logged_urls():
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql("SELECT url FROM malicious_urls", conn)
    conn.close()
    return df

# ================================
# Streamlit App
# ================================
def main():
    st.set_page_config("Malicious URL Detection", layout="centered")
    st.title("🛡️ Malicious URL Detection (Upload Your Model)")

    init_db()

    # Sidebar model uploader
    st.sidebar.title("⚙️ Load Trained Model")
    uploaded_model = st.sidebar.file_uploader("Upload your trained model (.pkl)", type=["pkl"])

    if uploaded_model is None:
        st.warning("Please upload your trained Random Forest model (.pkl) in the sidebar.")
        return

    model = joblib.load(BytesIO(uploaded_model.read()))

    tab1, tab2, tab3 = st.tabs(["🔍 Check URL", "📁 Upload CSV", "🧾 Logs"])

    # ========== Tab 1: Single URL ==========
    with tab1:
        url = st.text_input("Enter a URL:")
        if url:
            features = extract_features(url)
            pred = model.predict(features)[0]
            label = "🟥 Malicious" if pred == 1 else "🟩 Safe"
            st.subheader(f"Prediction: {label}")
            st.write(features)

            if pred == 1:
                insert_url(url)
                st.success("Stored in DB.")

    # ========== Tab 2: Bulk Upload ==========
    with tab2:
        uploaded = st.file_uploader("Upload CSV with 'url' column", key="csv_uploader")
        if uploaded:
            df = pd.read_csv(uploaded)
            if 'url' in df.columns:
                df['features'] = df['url'].apply(extract_features)
                X = pd.concat(df['features'].tolist(), axis=0).reset_index(drop=True)
                df['prediction'] = model.predict(X)
                df['label'] = df['prediction'].apply(lambda x: "Malicious" if x == 1 else "Safe")
                st.dataframe(df[['url', 'label']])
                malicious = df[df['prediction'] == 1]['url']
                for u in malicious:
                    insert_url(u)
                st.success(f"Stored {len(malicious)} malicious URLs.")

    # ========== Tab 3: Logs ==========
    with tab3:
        if st.button("Show Malicious URLs from DB"):
            logs = get_logged_urls()
            if logs.empty:
                st.info("No malicious URLs logged yet.")
            else:
                st.dataframe(logs)

if __name__ == "__main__":
    main()


2025-05-04 16:55:33.146 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-04 16:55:33.147 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-04 16:55:33.644 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-04 16:55:33.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-04 16:55:33.663 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-04 16:55:33.664 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-04 16:55:33.665 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn